In [35]:
# Dependencies and Setup
import requests
import json
import codecs
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import matplotlib as mpl
import gmaps
%matplotlib inline
# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)



In [36]:
# File to Load
price_csv = "Resources/PricePSQ.csv"
affordability_csv = "Resources/affordability_income.csv"
#regionID (used as primary key in other instances) is different in rental_data.csv
rental_csv = "Resources/rental_data.csv"
price_income_csv = "Resources/price_to_income.csv"

In [37]:
# Read Affordability Data
pricing_data = pd.read_csv(price_csv, encoding = "ISO-8859-1")
afford_data = pd.read_csv(affordability_csv, encoding = "ISO-8859-1")
rental_data = pd.read_csv(rental_csv, encoding = "ISO-8859-1")
price_income_data = pd.read_csv(price_income_csv, encoding = "ISO-8859-1")

In [38]:
#Find the cities with median income of 75000 (based on Princeton's study) or greater per family and clean the data 
#This dataframe will only show the income data for December of each year 
#Data shown is from 2010-2018
good_income = afford_data.loc[afford_data["2018-03"] >= 75000, \
              ["RegionID", "RegionName", "2010-03", "2011-03", "2012-03", "2013-03", \
               "2014-03", "2015-03", "2016-03", "2017-03", "2018-03"]]

#Print the top income cities
good_income.sort_values(by=['2018-03'], ascending=False).head()


,RegionID,RegionName,2010-03,2011-03,2012-03,2013-03,2014-03,2015-03,2016-03,2017-03,2018-03
34,395059,"San Jose, CA",83870.93,85531.04,87031.24,92039.31,94683.99,100127.20,107486.71,114456.82,121388.69900
11,395057,"San Francisco, CA",72999.45,73138.14,73940.77,78307.98,81617.52,87056.94,93638.40,99269.76,105089.13310
7,395209,"Washington, DC",86548.90,87700.10,88766.74,90491.36,90421.65,92518.66,95301.86,98291.83,101670.89510
58,394415,"Stamford, CT",79839.31,79087.63,80136.44,83637.51,84399.80,87465.58,89324.98,90162.37,92947.39091
297,394900,"Napa, CA",66492.32,65975.29,66280.66,70349.52,72378.97,73916.41,77324.19,83514.34,90300.24882


Price to Income for cities where income is greater than 75K

In [39]:
# Merge the income and housing price on RegionID
price_to_income = pd.merge(good_income, price_income_data, on='RegionID', how='left')

# PFilter the rows for which the column Index is "Price To Income"
income = price_to_income[price_to_income['Index'] == 'Price To Income']

# Pull the columns shown below and delete the rest
march_income = income[["RegionID", "RegionName_x", "2010_03", "2011_03", "2012_03", "2013_03", "2014_03", "2015_03", "2016_03", "2017_03", "2018_03"]]

# Change the name of the column
march_income_clean = march_income.rename(columns={"RegionName_x": "RegionName"})

# Sort the income data by year 2018 
march_income_clean.sort_values(by=['2018_03'], ascending=False).head()

,RegionID,RegionName,2010_03,2011_03,2012_03,2013_03,2014_03,2015_03,2016_03,2017_03,2018_03
58,395065,"Santa Cruz, CA",8.283408,7.675891,7.197588,7.716812,8.886340,9.532659,9.551430,9.830453,10.093040
27,395059,"San Jose, CA",6.944003,6.542654,6.414938,7.236039,8.022476,8.462236,8.684795,8.798078,9.993398
9,395057,"San Francisco, CA",7.248000,6.733833,6.415946,7.176791,8.099977,8.353154,8.524281,8.564542,8.864445
33,753924,"Urban Honolulu, HI",6.687904,6.722835,6.655560,6.753525,7.225041,7.325037,7.569094,8.009257,8.191807
61,394731,"Kahului, HI",6.765090,6.384182,5.892622,6.034405,6.694787,7.197414,7.239739,7.436942,7.826431


Rent to Income for cities where income is greater than 75K

In [40]:
# Merge the income and housing price on RegionID
rent_to_income = pd.merge(good_income, price_income_data, on='RegionID', how='left')

# Filter the rows for which the column Index is "Rent Affordability"
rent = price_to_income[price_to_income['Index'] == 'Rent Affordability']

# Filter the columns and show only the columns below
march_rent = rent[["RegionID", "RegionName_x", "2011_03", "2012_03", "2013_03", "2014_03", "2015_03", "2016_03", "2017_03", "2018_03"]]

# Change the name of the row "Region Name" 
march_rent_clean = march_rent.rename(columns={"RegionName_x": "RegionName"})

# Sort the Rent Affordibility data by year 2018
march_rent_clean.sort_values(by=['2018_03'], ascending=False).head()

,RegionID,RegionName,2011_03,2012_03,2013_03,2014_03,2015_03,2016_03,2017_03,2018_03
60,395065,"Santa Cruz, CA",0.378978,0.382593,0.388840,0.373962,0.446843,0.461148,0.446597,0.439384
41,395068,"Santa Rosa, CA",0.340196,0.338064,0.344411,0.360477,0.390388,0.405240,0.391123,0.402158
11,395057,"San Francisco, CA",0.377696,0.382198,0.368085,0.379624,0.417520,0.426876,0.402056,0.387872
19,395056,"San Diego, CA",0.392414,0.397406,0.385960,0.398791,0.404298,0.401531,0.394516,0.387446
1,394913,"New York, NY",0.363721,0.367102,0.364429,0.386650,0.404256,0.404667,0.385103,0.369670


Mortgage to Income for cities where income is greater than 75K

In [41]:
# Mortgage Affordability Data
# In New York, for example, the mortgage payment is 26% of total income for 2018-03. 
# However, this is just the mortgage payment and does not include taxes, insurance and other expenses related to home ownership

# Merge the income and housing price on RegionID
mortgage_to_income = pd.merge(good_income, price_income_data, on='RegionID', how='left')

# Filter the rows for which the column Index is "Mortgage Affordability"
mortgage = mortgage_to_income[mortgage_to_income['Index'] == 'Mortgage Affordability']

# Filter the columns and show only the columns below
march_mortgage = mortgage[["RegionID", "RegionName_x", "2010_03", "2011_03", "2012_03", "2013_03", "2014_03", "2015_03", "2016_03", "2017_03", "2018_03"]]

# Change the name of the row "Region Name" 
march_mortgage_clean = march_mortgage.rename(columns={"RegionName_x": "RegionName"})

# Sort the Mortgage Affordibility data by year 2018
march_mortgage_clean.sort_values(by=['2018_03'], ascending=False).head()

,RegionID,RegionName,2010_03,2011_03,2012_03,2013_03,2014_03,2015_03,2016_03,2017_03,2018_03
59,395065,"Santa Cruz, CA",0.425428,0.388402,0.327890,0.335560,0.424175,0.424853,0.421532,0.461497,0.487495
28,395059,"San Jose, CA",0.356637,0.331060,0.292236,0.314654,0.382940,0.377146,0.383285,0.413031,0.482683
10,395057,"San Francisco, CA",0.372250,0.340734,0.292282,0.312078,0.386640,0.372284,0.376201,0.402068,0.428154
34,753924,"Urban Honolulu, HI",0.343484,0.340177,0.303198,0.293672,0.344876,0.326463,0.334046,0.376000,0.395665
62,394731,"Kahului, HI",0.347448,0.323041,0.268442,0.262402,0.319565,0.320775,0.319511,0.349132,0.378018


In [42]:
# data1 = pricing_data[["RegionName"]]
# data2 = afford_data[["RegionName"]]
# data3 = rental_data[[ "RegionName"]]
# data4 = price_income_data[["RegionName"]]

# Fill NaN values and convert 
rating1 = pricing_data["2018-03"].astype('int64', copy=False)
rating2 = afford_data["2018-03"].astype(float)
rating3 = rental_data["2018_03"].astype('int64', copy=False)
rating4 = price_income_data["2018_03"].astype('int64', copy=False)

In [43]:
Location1 = [(36.974117, -122.030796), (37.338208, -121.886329), (37.77493, -122.419416), (21.276532, -157.825731), (20.889335, -156.472947)]
weights1 = [(10.093040),(9.993398),(8.864445 ), (8.191807), (7.826431)]

# Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(Location1, weights=weights1, 
                                 max_intensity=10,
                                 point_radius=0.1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [44]:
airport_df = pd.read_csv('../RealEstate_Project1/Airport_Output.csv')

# Store latitude and longitude in locations
locations = airport_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = airport_df["Airport Rating"].astype(float)
# BONUS
# Plost as a Hybrid map
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))